# 新闻文本分类

- 学习链接：https://github.com/datawhalechina/team-learning-nlp/tree/master/NewsTextClassification
- 比赛链接：[零基础入门NLP - 新闻文本分类 - 天池](https://tianchi.aliyun.com/competition/entrance/531810/introduction)

## Task1

https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.6.6406111aIKCSLV&postId=118252

任务一主要是理解赛题和数据，并没实际工作量。

数据下载解压之后，得到三个文件：

```
test_a.csv               211M
test_a_sample_submit.csv 98K
train_set.csv            840M
```

训练集和测试集都对字符进行了匿名处理，所以不用分词这一步。

训练数据有 20w 条，使用 `\t` 分隔，第一列为标签，第二列为文本。标签有 14 类，其对应关系为：

```
科技: 0
股票: 1
体育: 2
娱乐: 3
时政: 4
社会: 5
教育: 6
财经: 7
家居: 8
游戏: 9
房产: 10
时尚: 11
彩票: 12
星座: 13
```

## Task2

https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.9.6406111aIKCSLV&postId=118253

任务二需要完成以下作业：

1. 假设字符 3750，字符 900 和字符 648 是句子的标点符号，请分析赛题每篇新闻平均由多少个句子构成？
2. 统计每类新闻中出现次数对多的字符

### 文本长度

In [ ]:
import pandas as pd


train_df = pd.read_csv('/kaggle/input/train_set.csv', sep='\t')


train_df['text_len'] = train_df['text'].apply(lambda x: len(x.split()))
train_df['text_len'].describe()

可以看出，平均每篇新闻有 907 个字符，最短的有 2 个字符，最长的有 57921 个字符。

### 新闻类别分布

统计每类新闻的样本个数。样本分布不平均，最少的只有 908 个，最多的达到了 38918 个。

In [ ]:
train_df['label'].value_counts()

### Q1

对于问题一，需要把每篇文章的句子按标点符号切分后再计算句子个数，可直接正则模块切分。

In [ ]:
import re


def sentence_mean():
    train_df['sentences'] = train_df['text'].apply(lambda x: len([s for s in re.split(r'\b(?:3750|900|648)\b', x) if s]))
    return train_df['sentences'].mean()


sentence_mean()

可以看出，每篇新闻平均有 79 个句子。

### Q2

问题二需要先根据新闻类别分类后再统一数据，这里使用到了 `loc` 对列数据进行筛选。

In [ ]:
from collections import Counter


def max_char():
    res = []
    for label in range(14):
        c = Counter()
        df = train_df['text'].loc[train_df['label']==label].apply(lambda x: x.split())
        for news in df:
            c.update(Counter(news))
        res.append({label: c.most_common(1)[0]})
        print('新闻类别 {} 出现最多的字符为 {}，共出现 {} 次'.format(label, *c.most_common(1)[0]))
    return res


max_char()

## Task3

https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.12.6406111aIKCSLV&postId=118254

任务三需要完成以下作业：

1. 尝试改变TF-IDF的参数，并验证精度
2. 尝试使用其他机器学习模型，完成训练和验证

在做作业之前，先根据文档学习一遍。首先是文本的表示方法

### 文本的表示方法

在数据真正进入训练之前，我们需要将原始文本转化为数字或向量。在词向量出现之前，看下有哪些方法。

#### One-hot

> 这里的One-hot与数据挖掘任务中的操作是一致的，即将每一个单词使用一个离散的向量表示。具体将每个字/词编码一个索引，然后根据索引进行赋值。

概念还是比较容易理解的，即把语料库的词汇表中的每一个词都对应一个向量，向量的维度即为词汇表中词的个数，且向量中只有一位为 1，其余均为 0。

把例子用代码来展示一下。

参考 [sklearn.preprocessing.OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

corpus = [
    *'我 爱 北 京 天 安 门'.split(),
    *'我 喜 欢 上 海'.split()
]


label_enc = LabelEncoder()
onehot_enc = OneHotEncoder(sparse=False)

label_encoded = label_enc.fit_transform(corpus)

onehot_enc.fit_transform(label_encoded.reshape(len(label_encoded), 1))

#### Bag of Words

> Bag of Words（词袋表示），也称为Count Vectors，每个文档的字/词可以使用其出现次数来进行表示。

将示例代码运行看看结果。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

vectorizer = CountVectorizer()
vectorizer.fit_transform(corpus).toarray()

#### N-gram

> N-gram 与 Count Vectors 类似，不过加入了相邻单词组合成为新的单词，并进行计数。

代码展示与 Bag of Words 相似，只需修改下 `CountVectorizer` 的参数。

指定的两个参数中，`analyzer` 指定词或字符级别的 n-grams，这里我们使用 `char`，`ngram_range` 指定 n-grams 的上下界。

参考 [sklearn.feature_extraction.text.CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [ ]:
corpus = [
    '我爱北京天安门',
    '我喜欢上海'
]


ngram_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2))
ngram_vectorizer.fit_transform(corpus)
ngram_vectorizer.get_feature_names()

#### tf-idf

> tf-idf（英语：term frequency–inverse document frequency）是一种用于信息检索与文本挖掘的常用加权技术。tf-idf是一种统计方法，用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度。字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。

词频：

$$tf(t, d) = \frac{词 t 在文档 d 中出现的次数}{文档 d 的总次数}$$

逆文档频率：

$$idf(t, D) = \log{\frac{N}{\left| \left\{ d \in D : t \in d \right\} \right|}}$$

$N$ 表示文档数，$\left| \left\{ d \in D : t \in d \right\} \right|$ 表示包含词 t 的文档数。

而 tf-idf 则是两者的乘积：

$$tfidf(t, d, D) = tf(t, d) \cdot idf(t, D)$$

### 基于机器学习的文本分类

看下第一个例子，使用 `CountVectorizer` 表示文本，使用岭回归分类器进行分类：

```python
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score


# 读取数据
train_df = pd.read_csv('../data/train_set.csv', sep='\t', nrows=15000)

# 初始化词嵌入并载入数据进行学习
vectorizer = CountVectorizer(max_features=3000)
train_test = vectorizer.fit_transform(train_df['text'])

# 初始化分类器并对前一万条数据（训练集）进行训练
clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

# 对后 5000 条数据（测试集）进行预测并打分
val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))
```

运行得到的结果是 `0.7422037924439758`。

第二个例子使用 tf-idf 表示文本，同样使用岭回归分类器。主要代码改变如下，由 `CountVectorizer` 换为 `TfidfVectorizer`：

```python
tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
```

得到的结果为 `0.8721598830546126`。

### Q1

在改变 tf-idf 的参数前，我们可以借助 `help` 函数看下 `TfidfVectorizer` 有哪些参数，或直接看 [sklearn.feature_extraction.text.TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html?highlight=tfidfvectorizer)。

首先介绍几个主要相关的参数：

- `ngram_rane`: `n-grams` 的上下界
- `max_features`: 指定构建只包含词频出现前 `max_features` 次的词汇表

试下改变上面的一个或两个参数，看下结果。

In [ ]:
import functools


import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score


train_df = pd.read_csv('/kaggle/input/train_set.csv', sep='\t', nrows=15000)


@functools.lru_cache()
def get_score(ngram_range=(1, 3), max_features=3000):
    tfidf = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    train_test = tfidf.fit_transform(train_df['text'])

    clf = RidgeClassifier()
    clf.fit(train_test[:10000], train_df['label'].values[:10000])

    val_pred = clf.predict(train_test[10000:])
    score = f1_score(train_df['label'].values[10000:], val_pred, average='macro')
    print(f'{ngram_range}-{max_features}: {score}')
    return score

In [ ]:
import operator


ridge_scores = {}
for a in ((1, 3), (1, 4), (2, 4), (1, 5), (2, 5)):
    for b in (3000, 4000, 5000):
        ridge_scores[f'{a}-{b}'] = get_score(a, b)
print(max(ridge_scores.items(), key=operator.itemgetter(1)))

就目前微调的几个参数来看，效果最好的组合是 `ngram_range=(1, 5), max_features=5000`，得分为 `0.8853177666563177`。

### Q2

这里使用 SVM、决策树等模型进行训练与验证，模型参数保持默认。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier


@functools.lru_cache()
def get_clf_score(clf, clf_name):
    tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=3000)
    train_test = tfidf.fit_transform(train_df['text'])

    clf.fit(train_test[:10000], train_df['label'].values[:10000])

    val_pred = clf.predict(train_test[10000:])
    score = f1_score(train_df['label'].values[10000:], val_pred, average='macro')
    print(f'{clf_name}: {score}')
    return score


clfs = {
    'svm': SVC(),
    'decision_tree': DecisionTreeClassifier(),
    'k_neighbor': KNeighborsClassifier(),
    'random_forest': RandomForestClassifier(),
    'adaboost':AdaBoostClassifier(),
}

In [ ]:
clf_scores = {}
for clf_name, clf in clfs.items():
    clf_scores[clf_name] = get_clf_score(clf, clf_name)

print(clf_scores)

## Task4

[Task4 基于深度学习的文本分类 1](https://github.com/datawhalechina/team-learning-nlp/blob/master/NewsTextClassification/Task4%20%E5%9F%BA%E4%BA%8E%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0%E7%9A%84%E6%96%87%E6%9C%AC%E5%88%86%E7%B1%BB1.md)

本次任务有两个作业：

1. 阅读 fastText 的文档，尝试修改参数，得到更好的分数
2. 基于验证集的结果调整超参数，使得模型性能更优

这次我们学习 fastText 并对新闻分类。

在之前的任务三中，我们采用 tdidf 表示文本并使用机器学习算法进行新闻文本分类。如教学资料所说，前一章节的文本表示方法，存在一定的问题：

> 转换得到的向量维度很高，需要较长的训练实践；没有考虑单词与单词之间的关系，只是进行了统计。

fastText 可以用于学习词向量、文本分类，适用于该任务。

看了示例代码，代码结构与之前的训练代码差不多，但输入数据需要进行特殊处理，因为 fastText 的输入数据的标签需要有 `__label__` 前缀。

In [ ]:
import fasttext
import pandas as pd
from sklearn.metrics import f1_score


# 读取数据
train_df = pd.read_csv('/kaggle/input/train_set.csv', sep='\t', nrows=15000)
# 将数据转为 fastText 需要的格式
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
# 保存数据到文件
train_df[['text','label_ft']].iloc[:-5000].to_csv('train.csv', index=None, header=None, sep='\t')

model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
                                  verbose=2, minCount=1, epoch=25, loss="hs", thread=1)

val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
print(f1_score(train_df['label'].values[-5000:].astype(str), val_pred, average='macro'))  # 0.8231153757515691

### Q1

试下不同学习率下的得分。

In [ ]:
import functools
import operator


@functools.lru_cache()
def train_with_lr(lr):
    model = fasttext.train_supervised('train.csv', lr=lr, wordNgrams=2, 
                                      verbose=2, minCount=1, epoch=25, loss="hs", thread=1)

    val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
    score = f1_score(train_df['label'].values[-5000:].astype(str), val_pred, average='macro')
    print(lr, '-', score)
    return score


In [ ]:
fast_scores = {}

for lr in range(1, 11):
    lr = lr / 10
    fast_scores[lr] = train_with_lr(lr)
print(fast_scores)

max(fast_scores.items(), key=operator.itemgetter(1))

为了复现结果，把 `thread` 设置为 1。最终结果是在其他参数不变时，学习率为 1 效果最好。

### Q2

fastText 可以根据验证集自动调整超参数，我们试试看。

首先把数据且分一下。使用 `split` 命令即可，每个文件 1000 行。

In [ ]:
!wc -l train.csv
!split -l 1000 -d -a 1 --additional-suffix=.csv train.csv train_ 
!ls -alh

这里采用十折交叉法进行验证。

In [ ]:
import os


def get_train_and_validation(k):
    train_file, test_file = f'_train_{k}.csv', f'train_{k}.csv'
    if not os.path.exists(train_file):
        with open(train_file, 'w', encoding='utf-8') as tf:
            fns = [fn for fn in os.listdir() if fn.startswith('train_{k}')]
            for fn in fns:
                with open(fn, encoding='utf-8') as f:
                    tf.write(f.read())
    return train_file, test_file

In [ ]:
train, valid = get_train_and_validation(0)
# model = fasttext.train_supervised(input=train, autotuneValidationFile=valid)

然而似乎出了点问题，`train_supervised` 函数一运行服务端就挂了，只能放弃。

## Task5

[Task5 基于深度学习的文本分类 2](https://github.com/datawhalechina/team-learning-nlp/blob/master/NewsTextClassification/Task5%20%E5%9F%BA%E4%BA%8E%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0%E7%9A%84%E6%96%87%E6%9C%AC%E5%88%86%E7%B1%BB2.md)

本次学习涉及 Word2Vec、TextCNN、TextRNN 和 HAN，相关知识点较多。

Word2Vec 的基本思路是通过中心词预测上下文（Skip-grams）或通过上下文预测中心词（CBOW）。
在此思想之上，又有层次 softmax（Hierarchical softmax）和负采样（Nagative sampling）对训练过程进行优化。

TextCNN 和 TextRNN 分别使用 CNN（卷积神经网络）和 RNN（循环神经网络）进行文本特征抽取，可以用来解决文本分类问题。

> Hierarchical Attention Network for Document Classification(HAN) 基于层级注意力，在单词和句子级别分别编码并基于注意力获得文档的表示，然后经过Softmax进行分类。

这次作业如下：

1. 尝试通过 Word2Vec 训练词向量
2. 尝试使用 TextCNN、TextRNN 完成文本表示
3. 尝试使用 HAN 进行文本分类

TextCNN 和 TextRNN 的代码可参考 [graykode/nlp-tutorial](https://github.com/graykode/nlp-tutorial)。

### Q1

Word2Vec 可借助 `gensim` 来训练，输入数据不严谨地处理一下，代码比较简单。

In [1]:
import logging
import pandas as pd


from gensim.models.word2vec import Word2Vec


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

train_df = pd.read_csv('/kaggle/input/train_set.csv', sep='\t', nrows=15000)

In [2]:
def line_sentences():
    return train_df['text'].apply(lambda x: x.split())



def word2vec_model():
    sentences = line_sentences()
    return Word2Vec(sentences, hs=1, window=6, workers=8, size=100)


# model = word2vec_model()

### Q2

借助 [TextCNN-Torch.py](https://github.com/graykode/nlp-tutorial/blob/master/2-1.TextCNN/TextCNN-Torch.py) 的代码，稍微修改一下。

In [30]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

dtype = torch.FloatTensor

# Text-CNN Parameter
embedding_size = 2 # n-gram
sequence_length = 500
num_classes = 14  # 0 or 1
filter_sizes = [2, 2, 2] # n-gram window
num_filters = 3


def get_sentences_and_labels(start=0, end=None, length=sequence_length):
    sentences, labels = [], []
    if not end:
        end = len(train_df['text'])
    for s, l in zip(train_df['text'][start:end], train_df['label'][start:end]):
        if len(s.split()) > length:
            sentences.append(' '.join(s.split()[:length]))
            labels.append(l)
    return sentences, labels

sentences, labels = get_sentences_and_labels(0, -5000)


word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
vocab_size = len(word_dict)


class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()

        self.num_filters_total = num_filters * len(filter_sizes)
        self.W = nn.Parameter(torch.empty(vocab_size, embedding_size).uniform_(-1, 1)).type(dtype)
        self.Weight = nn.Parameter(torch.empty(self.num_filters_total, num_classes).uniform_(-1, 1)).type(dtype)
        self.Bias = nn.Parameter(0.1 * torch.ones([num_classes])).type(dtype)

    def forward(self, X):
        embedded_chars = self.W[X] # [batch_size, sequence_length, sequence_length]
        embedded_chars = embedded_chars.unsqueeze(1) # add channel(=1) [batch, channel(=1), sequence_length, embedding_size]

        pooled_outputs = []
        for filter_size in filter_sizes:
            # conv : [input_channel(=1), output_channel(=3), (filter_height, filter_width), bias_option]
            conv = nn.Conv2d(1, num_filters, (filter_size, embedding_size), bias=True)(embedded_chars)
            h = F.relu(conv)
            # mp : ((filter_height, filter_width))
            mp = nn.MaxPool2d((sequence_length - filter_size + 1, 1))
            # pooled : [batch_size(=6), output_height(=1), output_width(=1), output_channel(=3)]
            pooled = mp(h).permute(0, 3, 2, 1)
            pooled_outputs.append(pooled)

        h_pool = torch.cat(pooled_outputs, len(filter_sizes)) # [batch_size(=6), output_height(=1), output_width(=1), output_channel(=3) * 3]
        h_pool_flat = torch.reshape(h_pool, [-1, self.num_filters_total]) # [batch_size(=6), output_height * output_width * (output_channel * 3)]

        model = torch.mm(h_pool_flat, self.Weight) + self.Bias # [batch_size, num_classes]
        return model


In [31]:
model = TextCNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# inputs = sentences
inputs = []
for sen in sentences:
    inputs.append(np.asarray([word_dict[n] for n in sen.split()]))

targets = []
for out in labels:
    targets.append(out) # To using Torch Softmax Loss function

input_batch = Variable(torch.LongTensor(inputs))
target_batch = Variable(torch.LongTensor(targets))


# Training
for epoch in range(50):
    optimizer.zero_grad()
    output = model(input_batch)

    # output : [batch_size, num_classes], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

Epoch: 0010 cost = 3.646382
Epoch: 0020 cost = 3.568530
Epoch: 0030 cost = 3.472274
Epoch: 0040 cost = 3.461277
Epoch: 0050 cost = 2.798807


由于训练速度太慢了，这里只做了简单演示，偏差非常大。

In [33]:
test_labels, tests = [], []
for sen, l in zip(*get_sentences_and_labels(start=-5000)):
    try:
        tests.append(np.asarray([word_dict[n] for n in sen.split()]))
        test_labels.append(l)
    except KeyError:
        pass
test_batch = Variable(torch.LongTensor(tests))


val_pred = model(test_batch).data.max(1, keepdim=True)[1]

In [35]:
from sklearn.metrics import f1_score

f1_score(test_labels, val_pred, average='macro')

0.0012947777298230473